In [5]:
import data_processing
import numpy as np

In [2]:
text = data_processing.read_data()

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

In [6]:
encoded = np.array([char2int[ch] for ch in text])

In [8]:
encoded.shape

(1115394,)

In [10]:
data = np.lib.stride_tricks.sliding_window_view(encoded, window_shape=5, axis=0)

In [14]:
data.shape

(1115390, 5)

In [16]:
data[:int(len(data)*0.8), :].shape

(892312, 5)

In [20]:
def window_data_test_train(data_window,train_split=0.8):
    X_train = data_window[:int(len(data)*train_split)-1, :]
    X_test = data_window[int(len(data)*train_split)-1:, :]
    Y_train = data_window[1:int(len(data)*train_split), :]
    Y_test = data_window[1+int(len(data)*train_split):, :]
    return X_train, X_test, Y_train, Y_test

In [21]:
x_train,x_test,y_train,y_test = window_data_test_train(data)


In [22]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((892311, 5), (223079, 5), (892311, 5), (223077, 5))

In [24]:
y_train

array([[11, 49,  8, 23, 41],
       [49,  8, 23, 41, 24],
       [ 8, 23, 41, 24, 11],
       ...,
       [41, 23, 16, 45, 23],
       [23, 16, 45, 23, 41],
       [16, 45, 23, 41, 63]], shape=(892311, 5))

In [23]:
x_train

array([[10, 11, 49,  8, 23],
       [11, 49,  8, 23, 41],
       [49,  8, 23, 41, 24],
       ...,
       [ 0, 41, 23, 16, 45],
       [41, 23, 16, 45, 23],
       [23, 16, 45, 23, 41]], shape=(892311, 5))